# Sankey diagram

In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

## Preparations

In [4]:
# Load the data
project = pd.read_excel("C:/Users/sarah/OneDrive - KU Leuven/Desktop/Modern Data Analytics/Project/data/project.xlsx")
organization = pd.read_excel("C:/Users/sarah/OneDrive - KU Leuven/Desktop/Modern Data Analytics/Project/data/organization.xlsx")
esv = pd.read_excel("C:/Users/sarah/OneDrive - KU Leuven/Desktop/Modern Data Analytics/Project/data/euroSciVoc.xlsx")

# Merge the dataframes
df = project.merge(organization, left_on='id', right_on='projectID', how='inner')
df = df.merge(esv, left_on='id', right_on='projectID', how='inner')

# Drop redundant projectID columns
df = df.drop(columns=['projectID_x', 'projectID_y'])

# Set datatypes
df['ecMaxContribution'] = pd.to_numeric(df['ecMaxContribution'], errors='coerce')

# Ensure the 'euroSciVocPath' column is treated as a string and handle NaN values
df['euroSciVocPath'] = df['euroSciVocPath'].fillna('').astype(str)

# Ensure proper formatting of 'city' column
df['city'] = df['city'].str.title()
df['city'] = df['city'].replace({
    'Paris 15': 'Paris',
    'Thermi Thessaloniki': 'Thessaloniki',
    'Athina': 'Athens'
})

# Display the first few rows of the merged dataframe
print(df.columns)

Index(['id', 'status', 'title', 'startDate', 'endDate', 'totalCost_x',
       'ecMaxContribution', 'ecSignatureDate', 'organisationID', 'name',
       'activityType', 'city', 'country', 'geolocation', 'order', 'role',
       'ecContribution', 'netEcContribution', 'totalCost_y',
       'endOfParticipation', 'euroSciVocCode', 'euroSciVocPath',
       'euroSciVocTitle'],
      dtype='object')


In [5]:
# Replace empty strings with a placeholder (if needed)
df['euroSciVocPath'] = df['euroSciVocPath'].replace('', 'Unknown')

# Extract the first part of the 'euroSciVocPath' column and store it in a new column called 'field'
df['field'] = df['euroSciVocPath'].str.extract(r'^/?([^/]+)')

# Display the first few rows to verify
print(df[['euroSciVocPath', 'field']].head())
print(df['field'].unique())

                                      euroSciVocPath                  field
0  /social sciences/political sciences/government...        social sciences
1  /agricultural sciences/agriculture, forestry, ...  agricultural sciences
2  /natural sciences/physical sciences/optics/mic...       natural sciences
3  /natural sciences/physical sciences/astronomy/...       natural sciences
4                               /social sciences/law        social sciences
['social sciences' 'agricultural sciences' 'natural sciences'
 'engineering and technology' 'medical and health sciences' 'humanities']


## Country

In [4]:
# Get money percentages per country
total_money = df['ecMaxContribution'].sum()
country_money = (df.groupby('country')['ecMaxContribution'].sum() / total_money) * 100

In [5]:
# Convert the series to a dataframe
country_money_df = country_money.reset_index()
country_money_df.columns = ['Country', 'Money']

# Create a pie chart
fig = px.pie(country_money_df, values='Money', names='Country', title='Country Money Pie Chart')

# Display the pie chart
fig.show()

In [6]:
# Convert the series to a dataframe
country_money_df = country_money.reset_index()
country_money_df.columns = ['Country', 'Money']

# Filter for countries with more than 3% of total money
filtered_country_money = country_money[country_money > 3]
filtered_countries = filtered_country_money.index

# Classify countries not in the filtered list as 'Other'
country_money_df['Country'] = country_money_df['Country'].apply(
    lambda x: x if x in filtered_countries else 'Other'
)

# Regroup after merging to 'Other'
country_money_df = country_money_df.groupby('Country', as_index=False).sum()

# Display the resulting DataFrame
country_money_df

,Country,Money
0,BE,5.353291
1,DE,9.715996
2,EL,4.840373
3,ES,11.391466
4,FR,10.151557
5,IT,9.071496
6,NL,5.568877
7,Other,36.930057
8,SE,3.234278
9,UK,3.731761


In [7]:
# Create a new DataFrame with the specified structure
sankey_country = pd.DataFrame({
    'source': ['ecMaxContribution'] * (country_money_df.shape[0]),
    'target': country_money_df['Country'],
    'value': country_money_df['Money']
})

# Create a list of unique labels
labels = ['ecMaxContribution'] + country_money_df['Country'].tolist()

# Map source and target columns to indices in the labels list
sankey_country['source'] = sankey_country['source'].map(lambda x: labels.index(x))
sankey_country['target'] = sankey_country['target'].map(lambda x: labels.index(x))

In [ ]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sankey_country['source'],
        target=sankey_country['target'],
        value=sankey_country['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution (Country)", font_size=10)
fig.show()

## Field

In [9]:
# Get money percentages per field
total_money = df['ecMaxContribution'].sum()
field_money = (df.groupby('field')['ecMaxContribution'].sum() / total_money) * 100

In [10]:
# Convert the series to a dataframe
field_money_df = field_money.reset_index()
field_money_df.columns = ['Field', 'Money']

# Create a pie chart
fig = px.pie(field_money_df, values='Money', names='Field', title='Field Money Pie Chart')

# Display the pie chart
fig.show()

In [ ]:
# Create a new DataFrame with the specified structure
sankey_field = pd.DataFrame({
    'source': ['ecMaxContribution'] * (field_money_df.shape[0]),
    'target': field_money_df['Field'],
    'value': field_money_df['Money']
})

# Create a list of unique labels 
labels = ['ecMaxContribution'] + field_money_df['Field'].tolist()

# Map source and target columns to indices in the labels list
sankey_field['source'] = sankey_field['source'].map(lambda x: labels.index(x))
sankey_field['target'] = sankey_field['target'].map(lambda x: labels.index(x))

In [ ]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sankey_field['source'],
        target=sankey_field['target'],
        value=sankey_field['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution (Field)", font_size=10)
fig.show()

## Activity Type

In [29]:
# Get money percentages per activity type
total_money = df['ecMaxContribution'].sum()
activity_money = (df.groupby('activityType')['ecMaxContribution'].sum() / total_money) * 100

In [31]:
# Convert the series to a dataframe
activity_money_df = activity_money.reset_index()
activity_money_df.columns = ['Activity Type', 'Money']

# Create a pie chart
fig = px.pie(activity_money_df, values='Money', names='Activity Type', title='Activity Type Money Pie Chart')

# Display the pie chart
fig.show()

In [33]:
# Create a new DataFrame with the specified structure
sankey_field = pd.DataFrame({
    'source': ['ecMaxContribution'] * (activity_money_df.shape[0]),
    'target': activity_money_df['Activity Type'],
    'value': activity_money_df['Money']
})

# Create a list of unique labels 
labels = ['ecMaxContribution'] + activity_money_df['Activity Type'].tolist()

# Map source and target columns to indices in the labels list
sankey_field['source'] = sankey_field['source'].map(lambda x: labels.index(x))
sankey_field['target'] = sankey_field['target'].map(lambda x: labels.index(x))

In [ ]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=sankey_field['source'],
        target=sankey_field['target'],
        value=sankey_field['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution (Activity Type)", font_size=10)
fig.show()

## Country ➝ Field

In [6]:
# Get money percentages per country and field
total_money = df['ecMaxContribution'].sum()
country_money = (df.groupby('country')['ecMaxContribution'].sum() / total_money) * 100
country_field_money = (df.groupby(['country', 'field'])['ecMaxContribution'].sum() / total_money) * 100

In [7]:
# Convert the country series to a dataframe
country_money_df = country_money.reset_index()
country_money_df.columns = ['Country', 'Money']

# Filter for countries with more than 3% of total money
filtered_country_money = country_money[country_money > 3]  
filtered_countries = filtered_country_money.index

# Convert the country-field series to a dataframe
country_field_money_df = country_field_money.reset_index()
country_field_money_df.columns = ['Country', 'Field', 'Money']

# Classify countries not in the filtered list as 'Other'
country_field_money_df['Country'] = country_field_money_df['Country'].apply(
    lambda x: x if x in filtered_countries else 'Other'
)

# Regroup after merging to 'Other'
country_field_money_df = country_field_money_df.groupby(['Country', 'Field'], as_index=False).sum()

In [8]:
# Create a list of unique labels 
labels = ['ecMaxContribution'] + country_field_money_df['Country'].unique().tolist() + country_field_money_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ Country
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * country_field_money_df['Country'].nunique(),
    'target': country_field_money_df['Country'].unique(),
    'value': country_field_money_df.groupby('Country')['Money'].sum().values
})

# 2: Country ➝ Field
links2 = pd.DataFrame({
    'source': country_field_money_df['Country'],
    'target': country_field_money_df['Field'],
    'value': country_field_money_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [9]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution (Country ➝ Field)", font_size=10)
fig.show()

## Country ➝ Field ➝ Activity Type

In [19]:
# Get money percentages per country, field and activity type
total_money = df['ecMaxContribution'].sum()
country_money = (df.groupby('country')['ecMaxContribution'].sum() / total_money) * 100
country_field_money = (df.groupby(['country', 'field'])['ecMaxContribution'].sum() / total_money) * 100
field_activity_money = (df.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money) * 100
country_field_activity_money = (df.groupby(['country', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money) * 100

In [20]:
# Convert the country series to a dataframe
country_money_df = country_money.reset_index()
country_money_df.columns = ['Country', 'Money']

# Filter for countries with more than 3% of total money
filtered_country_money = country_money[country_money > 3]  
filtered_countries = filtered_country_money.index

# Convert the country-field series to a dataframe
country_field_money_df = country_field_money.reset_index()
country_field_money_df.columns = ['Country', 'Field', 'Money']

# Classify countries not in the filtered list as 'Other'
country_field_money_df['Country'] = country_field_money_df['Country'].apply(
    lambda x: x if x in filtered_countries else 'Other'
)

# Regroup after merging to 'Other'
country_field_money_df = country_field_money_df.groupby(['Country', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_df = field_activity_money.reset_index()
field_activity_money_df.columns = ['Field', 'ActivityType', 'Money']

In [21]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + country_field_money_df['Country'].unique().tolist() + country_field_money_df['Field'].unique().tolist() + field_activity_money_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create the first set of links: ecMaxContribution ➝ Country
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * country_field_money_df['Country'].nunique(),
    'target': country_field_money_df['Country'].unique(),
    'value': country_field_money_df.groupby('Country')['Money'].sum().values
})

# Create the second set of links: Country ➝ Field
links2 = pd.DataFrame({
    'source': country_field_money_df['Country'],
    'target': country_field_money_df['Field'],
    'value': country_field_money_df['Money']
})

# Create the third set of links: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_df['Field'],
    'target': field_activity_money_df['ActivityType'],
    'value': field_activity_money_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [22]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution (Country ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## ES

### City ➝ Field

In [ ]:
# Filter for ES
df_ES = df[df['country'] == 'ES']

# Get money percentages per city of ES and field
total_money_ES = df_ES['ecMaxContribution'].sum()
city_money_ES = (df_ES.groupby('city')['ecMaxContribution'].sum() / total_money_ES) * 100
city_field_money_ES = (df_ES.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_ES) * 100

In [ ]:
# Convert the city series to a dataframe
city_money_ES_df = city_money_ES.reset_index()
city_money_ES_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_ES = city_money_ES[city_money_ES > 3]  
filtered_cities_ES = filtered_city_money_ES.index

# Convert the city-field series to a dataframe
city_field_money_ES_df = city_field_money_ES.reset_index()
city_field_money_ES_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_ES_df['City'] = city_field_money_ES_df['City'].apply(
    lambda x: x if x in filtered_cities_ES else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_ES_df = city_field_money_ES_df.groupby(['City', 'Field'], as_index=False).sum()

In [12]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_ES_df['City'].unique().tolist() + city_field_money_ES_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_ES_df['City'].nunique(),
    'target': city_field_money_ES_df['City'].unique(),
    'value': city_field_money_ES_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_ES_df['City'],
    'target': city_field_money_ES_df['Field'],
    'value': city_field_money_ES_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [13]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of ES (City ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [ ]:
# Filter for ES
df_ES = df[df['country'] == 'ES']

# Get money percentages per city of ES and field
total_money_ES = df_ES['ecMaxContribution'].sum()
city_money_ES = (df_ES.groupby('city')['ecMaxContribution'].sum() / total_money_ES) * 100
city_field_money_ES = (df_ES.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_ES) * 100
field_activity_money_ES = (df_ES.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_ES) * 100
city_field_activity_money_ES = (df_ES.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_ES) * 100

In [28]:
# Convert the city series to a dataframe
city_money_ES_df = city_money_ES.reset_index()
city_money_ES_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_ES = city_money_ES[city_money_ES > 3]  
filtered_cities_ES = filtered_city_money_ES.index

# Convert the city-field series to a dataframe
city_field_money_ES_df = city_field_money_ES.reset_index()
city_field_money_ES_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_ES_df['City'] = city_field_money_ES_df['City'].apply(
    lambda x: x if x in filtered_cities_ES else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_ES_df = city_field_money_ES_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_ES_df = field_activity_money_ES.reset_index()
field_activity_money_ES_df.columns = ['Field', 'ActivityType', 'Money']

In [29]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_ES_df['City'].unique().tolist() + city_field_money_ES_df['Field'].unique().tolist() + field_activity_money_ES_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_ES_df['City'].nunique(),
    'target': city_field_money_ES_df['City'].unique(),
    'value': city_field_money_ES_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_ES_df['City'],
    'target': city_field_money_ES_df['Field'],
    'value': city_field_money_ES_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_ES_df['Field'],
    'target': field_activity_money_ES_df['ActivityType'],
    'value': field_activity_money_ES_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [30]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of ES (Country ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## FR

### City ➝ Field

In [42]:
# Filter for FR
df_FR = df[df['country'] == 'FR']

# Get money percentages per city of FR and field
total_money_FR = df_FR['ecMaxContribution'].sum()
city_money_FR = (df_FR.groupby('city')['ecMaxContribution'].sum() / total_money_FR) * 100
city_field_money_FR = (df_FR.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_FR) * 100

In [ ]:
# Convert the city series to a dataframe
city_money_FR_df = city_money_FR.reset_index()
city_money_FR_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_FR = city_money_FR[city_money_FR > 3]  
filtered_cities_FR = filtered_city_money_FR.index

# Convert the city-field series to a dataframe
city_field_money_FR_df = city_field_money_FR.reset_index()
city_field_money_FR_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_FR_df['City'] = city_field_money_FR_df['City'].apply(
    lambda x: x if x in filtered_cities_FR else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_FR_df = city_field_money_FR_df.groupby(['City', 'Field'], as_index=False).sum()

In [ ]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_FR_df['City'].unique().tolist() + city_field_money_FR_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_FR_df['City'].nunique(),
    'target': city_field_money_FR_df['City'].unique(),
    'value': city_field_money_FR_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_FR_df['City'],
    'target': city_field_money_FR_df['Field'],
    'value': city_field_money_FR_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [ ]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of FR (Country ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [50]:
# Filter for FR
df_FR = df[df['country'] == 'FR']

# Get money percentages per city of FR and field
total_money_FR = df_FR['ecMaxContribution'].sum()
city_money_FR = (df_FR.groupby('city')['ecMaxContribution'].sum() / total_money_FR) * 100
city_field_money_FR = (df_FR.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_FR) * 100
field_activity_money_FR = (df_FR.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_FR) * 100
city_field_activity_money_FR = (df_FR.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_FR) * 100

In [51]:
# Convert the city series to a dataframe
city_money_FR_df = city_money_FR.reset_index()
city_money_FR_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_FR = city_money_FR[city_money_FR > 3]  
filtered_cities_FR = filtered_city_money_FR.index

# Convert the city-field series to a dataframe
city_field_money_FR_df = city_field_money_FR.reset_index()
city_field_money_FR_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_FR_df['City'] = city_field_money_FR_df['City'].apply(
    lambda x: x if x in filtered_cities_FR else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_FR_df = city_field_money_FR_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_FR_df = field_activity_money_FR.reset_index()
field_activity_money_FR_df.columns = ['Field', 'ActivityType', 'Money']

In [54]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_FR_df['City'].unique().tolist() + city_field_money_FR_df['Field'].unique().tolist() + field_activity_money_FR_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_FR_df['City'].nunique(),
    'target': city_field_money_FR_df['City'].unique(),
    'value': city_field_money_FR_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_FR_df['City'],
    'target': city_field_money_FR_df['Field'],
    'value': city_field_money_FR_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_FR_df['Field'],
    'target': field_activity_money_FR_df['ActivityType'],
    'value': field_activity_money_FR_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [ ]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of FR (City ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## EL

### City ➝ Field

In [61]:
# Filter for EL
df_EL = df[df['country'] == 'EL']

# Get money percentages per city of EL and field
total_money_EL = df_EL['ecMaxContribution'].sum()
city_money_EL = (df_EL.groupby('city')['ecMaxContribution'].sum() / total_money_EL) * 100
city_field_money_EL = (df_EL.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_EL) * 100

In [62]:
# Convert the city series to a dataframe
city_money_EL_df = city_money_EL.reset_index()
city_money_EL_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_EL = city_money_EL[city_money_EL > 3]  
filtered_cities_EL = filtered_city_money_EL.index

# Convert the city-field series to a dataframe
city_field_money_EL_df = city_field_money_EL.reset_index()
city_field_money_EL_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_EL_df['City'] = city_field_money_EL_df['City'].apply(
    lambda x: x if x in filtered_cities_EL else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_EL_df = city_field_money_EL_df.groupby(['City', 'Field'], as_index=False).sum()

In [63]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_EL_df['City'].unique().tolist() + city_field_money_EL_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_EL_df['City'].nunique(),
    'target': city_field_money_EL_df['City'].unique(),
    'value': city_field_money_EL_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_EL_df['City'],
    'target': city_field_money_EL_df['Field'],
    'value': city_field_money_EL_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [64]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of EL (City ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [122]:
# Filter for EL
df_EL = df[df['country'] == 'EL']

# Get money percentages per city of EL and field
total_money_EL = df_EL['ecMaxContribution'].sum()
city_money_EL = (df_EL.groupby('city')['ecMaxContribution'].sum() / total_money_EL) * 100
city_field_money_EL = (df_EL.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_EL) * 100
field_activity_money_EL = (df_EL.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_EL) * 100
city_field_activity_money_EL = (df_EL.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_EL) * 100

In [123]:
# Convert the city series to a dataframe
city_money_EL_df = city_money_EL.reset_index()
city_money_EL_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_EL = city_money_EL[city_money_EL > 3]  
filtered_cities_EL = filtered_city_money_EL.index

# Convert the city-field series to a dataframe
city_field_money_EL_df = city_field_money_EL.reset_index()
city_field_money_EL_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_EL_df['City'] = city_field_money_EL_df['City'].apply(
    lambda x: x if x in filtered_cities_EL else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_EL_df = city_field_money_EL_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_EL_df = field_activity_money_EL.reset_index()
field_activity_money_EL_df.columns = ['Field', 'ActivityType', 'Money']

In [124]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_EL_df['City'].unique().tolist() + city_field_money_EL_df['Field'].unique().tolist() + field_activity_money_EL_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_EL_df['City'].nunique(),
    'target': city_field_money_EL_df['City'].unique(),
    'value': city_field_money_EL_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_EL_df['City'],
    'target': city_field_money_EL_df['Field'],
    'value': city_field_money_EL_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_FR_df['Field'],
    'target': field_activity_money_FR_df['ActivityType'],
    'value': field_activity_money_FR_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [125]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of EL (City ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## IT

### City ➝ Field

In [69]:
# Filter for IT
df_IT = df[df['country'] == 'IT']

# Get money percentages per city of IT and field
total_money_IT = df_IT['ecMaxContribution'].sum()
city_money_IT = (df_IT.groupby('city')['ecMaxContribution'].sum() / total_money_IT) * 100
city_field_money_IT = (df_IT.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_IT) * 100

In [70]:
# Convert the city series to a dataframe
city_money_IT_df = city_money_IT.reset_index()
city_money_IT_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_IT = city_money_IT[city_money_IT > 3]  
filtered_cities_IT = filtered_city_money_IT.index

# Convert the city-field series to a dataframe
city_field_money_IT_df = city_field_money_IT.reset_index()
city_field_money_IT_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_IT_df['City'] = city_field_money_IT_df['City'].apply(
    lambda x: x if x in filtered_cities_IT else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_IT_df = city_field_money_IT_df.groupby(['City', 'Field'], as_index=False).sum()

In [71]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_IT_df['City'].unique().tolist() + city_field_money_IT_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_IT_df['City'].nunique(),
    'target': city_field_money_IT_df['City'].unique(),
    'value': city_field_money_IT_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_IT_df['City'],
    'target': city_field_money_IT_df['Field'],
    'value': city_field_money_IT_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [72]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of IT (City ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [73]:
# Filter for IT
df_IT = df[df['country'] == 'IT']

# Get money percentages per city of IT and field
total_money_IT = df_IT['ecMaxContribution'].sum()
city_money_IT = (df_IT.groupby('city')['ecMaxContribution'].sum() / total_money_IT) * 100
city_field_money_IT = (df_IT.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_IT) * 100
field_activity_money_IT = (df_IT.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_IT) * 100
city_field_activity_money_IT = (df_IT.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_IT) * 100

In [74]:
# Convert the city series to a dataframe
city_money_IT_df = city_money_IT.reset_index()
city_money_IT_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_IT = city_money_IT[city_money_IT > 3]  
filtered_cities_IT = filtered_city_money_IT.index

# Convert the city-field series to a dataframe
city_field_money_IT_df = city_field_money_IT.reset_index()
city_field_money_IT_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_IT_df['City'] = city_field_money_IT_df['City'].apply(
    lambda x: x if x in filtered_cities_IT else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_IT_df = city_field_money_IT_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_IT_df = field_activity_money_IT.reset_index()
field_activity_money_IT_df.columns = ['Field', 'ActivityType', 'Money']

In [75]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_IT_df['City'].unique().tolist() + city_field_money_IT_df['Field'].unique().tolist() + field_activity_money_IT_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_IT_df['City'].nunique(),
    'target': city_field_money_IT_df['City'].unique(),
    'value': city_field_money_IT_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_IT_df['City'],
    'target': city_field_money_IT_df['Field'],
    'value': city_field_money_IT_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_FR_df['Field'],
    'target': field_activity_money_FR_df['ActivityType'],
    'value': field_activity_money_FR_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [ ]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of IT (City ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## DE

### City ➝ Field

In [ ]:
# Filter for DE
df_DE = df[df['country'] == 'DE']

# Get money percentages per city of DE and field
total_money_DE = df_DE['ecMaxContribution'].sum()
city_money_DE = (df_DE.groupby('city')['ecMaxContribution'].sum() / total_money_DE) * 100
city_field_money_DE = (df_DE.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_DE) * 100

In [85]:
# Convert the city series to a dataframe
city_money_DE_df = city_money_DE.reset_index()
city_money_DE_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_DE = city_money_DE[city_money_DE > 3]  
filtered_cities_DE = filtered_city_money_DE.index

# Convert the city-field series to a dataframe
city_field_money_DE_df = city_field_money_DE.reset_index()
city_field_money_DE_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_DE_df['City'] = city_field_money_DE_df['City'].apply(
    lambda x: x if x in filtered_cities_DE else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_DE_df = city_field_money_DE_df.groupby(['City', 'Field'], as_index=False).sum()

In [86]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_DE_df['City'].unique().tolist() + city_field_money_DE_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_DE_df['City'].nunique(),
    'target': city_field_money_DE_df['City'].unique(),
    'value': city_field_money_DE_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_DE_df['City'],
    'target': city_field_money_DE_df['Field'],
    'value': city_field_money_DE_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [87]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of DE (City ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [118]:
# Filter for DE
df_DE = df[df['country'] == 'DE']

# Get money percentages per city of DE and field
total_money_DE = df_DE['ecMaxContribution'].sum()
city_money_DE = (df_DE.groupby('city')['ecMaxContribution'].sum() / total_money_DE) * 100
city_field_money_DE = (df_DE.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_DE) * 100
field_activity_money_DE = (df_DE.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_DE) * 100
city_field_activity_money_DE = (df_DE.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_DE) * 100

In [119]:
# Convert the city series to a dataframe
city_money_DE_df = city_money_DE.reset_index()
city_money_DE_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_DE = city_money_DE[city_money_DE > 3]  
filtered_cities_DE = filtered_city_money_DE.index

# Convert the city-field series to a dataframe
city_field_money_DE_df = city_field_money_DE.reset_index()
city_field_money_DE_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_DE_df['City'] = city_field_money_DE_df['City'].apply(
    lambda x: x if x in filtered_cities_DE else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_DE_df = city_field_money_DE_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_DE_df = field_activity_money_DE.reset_index()
field_activity_money_DE_df.columns = ['Field', 'ActivityType', 'Money']

In [120]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_DE_df['City'].unique().tolist() + city_field_money_DE_df['Field'].unique().tolist() + field_activity_money_DE_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_DE_df['City'].nunique(),
    'target': city_field_money_DE_df['City'].unique(),
    'value': city_field_money_DE_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_DE_df['City'],
    'target': city_field_money_DE_df['Field'],
    'value': city_field_money_DE_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_FR_df['Field'],
    'target': field_activity_money_FR_df['ActivityType'],
    'value': field_activity_money_FR_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [121]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of DE (City ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## BE

### City ➝ Field

In [ ]:
# Filter for BE
df_BE = df[df['country'] == 'BE']

# Get money percentages per city of BE and field
total_money_BE = df_BE['ecMaxContribution'].sum()
city_money_BE = (df_BE.groupby('city')['ecMaxContribution'].sum() / total_money_BE) * 100
city_field_money_BE = (df_BE.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_BE) * 100

In [102]:
# Convert the city series to a dataframe
city_money_BE_df = city_money_BE.reset_index()
city_money_BE_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_BE = city_money_BE[city_money_BE > 3]  
filtered_cities_BE = filtered_city_money_BE.index

# Convert the city-field series to a dataframe
city_field_money_BE_df = city_field_money_BE.reset_index()
city_field_money_BE_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_BE_df['City'] = city_field_money_BE_df['City'].apply(
    lambda x: x if x in filtered_cities_BE else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_BE_df = city_field_money_BE_df.groupby(['City', 'Field'], as_index=False).sum()

In [ ]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_BE_df['City'].unique().tolist() + city_field_money_BE_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_BE_df['City'].nunique(),
    'target': city_field_money_BE_df['City'].unique(),
    'value': city_field_money_BE_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_BE_df['City'],
    'target': city_field_money_BE_df['Field'],
    'value': city_field_money_BE_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [104]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of BE (City ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [91]:
# Filter for BE
df_BE = df[df['country'] == 'BE']

# Get money percentages per city of BE and field
total_money_BE = df_BE['ecMaxContribution'].sum()
city_money_BE = (df_BE.groupby('city')['ecMaxContribution'].sum() / total_money_BE) * 100
city_field_money_BE = (df_BE.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_BE) * 100
field_activity_money_BE = (df_BE.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_BE) * 100
city_field_activity_money_BE = (df_BE.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_BE) * 100

In [92]:
# Convert the city series to a dataframe
city_money_BE_df = city_money_BE.reset_index()
city_money_BE_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_BE = city_money_BE[city_money_BE > 3]  
filtered_cities_BE = filtered_city_money_BE.index

# Convert the city-field series to a dataframe
city_field_money_BE_df = city_field_money_BE.reset_index()
city_field_money_BE_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_BE_df['City'] = city_field_money_BE_df['City'].apply(
    lambda x: x if x in filtered_cities_BE else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_BE_df = city_field_money_BE_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_BE_df = field_activity_money_BE.reset_index()
field_activity_money_BE_df.columns = ['Field', 'ActivityType', 'Money']

In [93]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_BE_df['City'].unique().tolist() + city_field_money_BE_df['Field'].unique().tolist() + field_activity_money_BE_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_BE_df['City'].nunique(),
    'target': city_field_money_BE_df['City'].unique(),
    'value': city_field_money_BE_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_BE_df['City'],
    'target': city_field_money_BE_df['Field'],
    'value': city_field_money_BE_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_BE_df['Field'],
    'target': field_activity_money_BE_df['ActivityType'],
    'value': field_activity_money_BE_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [94]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of BE (City ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## SE

### City ➝ Field

In [96]:
# Filter for SE
df_SE = df[df['country'] == 'SE']

# Get money percentages per city of SE and field
total_money_SE = df_SE['ecMaxContribution'].sum()
city_money_SE = (df_SE.groupby('city')['ecMaxContribution'].sum() / total_money_SE) * 100
city_field_money_SE = (df_SE.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_SE) * 100

In [97]:
# Convert the city series to a dataframe
city_money_SE_df = city_money_SE.reset_index()
city_money_SE_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_SE = city_money_SE[city_money_SE > 3]  
filtered_cities_SE = filtered_city_money_SE.index

# Convert the city-field series to a dataframe
city_field_money_SE_df = city_field_money_SE.reset_index()
city_field_money_SE_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_SE_df['City'] = city_field_money_SE_df['City'].apply(
    lambda x: x if x in filtered_cities_SE else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_SE_df = city_field_money_SE_df.groupby(['City', 'Field'], as_index=False).sum()

In [98]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_SE_df['City'].unique().tolist() + city_field_money_SE_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_SE_df['City'].nunique(),
    'target': city_field_money_SE_df['City'].unique(),
    'value': city_field_money_SE_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_SE_df['City'],
    'target': city_field_money_SE_df['Field'],
    'value': city_field_money_SE_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [99]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of SE (City ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [100]:
# Filter for SE
df_SE = df[df['country'] == 'SE']

# Get money percentages per city of SE and field
total_money_SE = df_SE['ecMaxContribution'].sum()
city_money_SE = (df_SE.groupby('city')['ecMaxContribution'].sum() / total_money_SE) * 100
city_field_money_SE = (df_SE.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_SE) * 100
field_activity_money_SE = (df_SE.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_SE) * 100
city_field_activity_money_SE = (df_SE.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_SE) * 100

In [101]:
# Convert the city series to a dataframe
city_money_SE_df = city_money_SE.reset_index()
city_money_SE_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_SE = city_money_SE[city_money_SE > 3]  
filtered_cities_SE = filtered_city_money_SE.index

# Convert the city-field series to a dataframe
city_field_money_SE_df = city_field_money_SE.reset_index()
city_field_money_SE_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_SE_df['City'] = city_field_money_SE_df['City'].apply(
    lambda x: x if x in filtered_cities_SE else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_SE_df = city_field_money_SE_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_SE_df = field_activity_money_SE.reset_index()
field_activity_money_SE_df.columns = ['Field', 'ActivityType', 'Money']

In [102]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_SE_df['City'].unique().tolist() + city_field_money_SE_df['Field'].unique().tolist() + field_activity_money_SE_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_SE_df['City'].nunique(),
    'target': city_field_money_SE_df['City'].unique(),
    'value': city_field_money_SE_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_SE_df['City'],
    'target': city_field_money_SE_df['Field'],
    'value': city_field_money_SE_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_SE_df['Field'],
    'target': field_activity_money_SE_df['ActivityType'],
    'value': field_activity_money_SE_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [103]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of SE (City ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## NL

### City ➝ Field

In [104]:
# Filter for NL
df_NL = df[df['country'] == 'NL']

# Get money percentages per city of NL and field
total_money_NL = df_NL['ecMaxContribution'].sum()
city_money_NL = (df_NL.groupby('city')['ecMaxContribution'].sum() / total_money_NL) * 100
city_field_money_NL = (df_NL.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_NL) * 100

In [105]:
# Convert the city series to a dataframe
city_money_NL_df = city_money_NL.reset_index()
city_money_NL_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_NL = city_money_NL[city_money_NL > 3]  
filtered_cities_NL = filtered_city_money_NL.index

# Convert the city-field series to a dataframe
city_field_money_NL_df = city_field_money_NL.reset_index()
city_field_money_NL_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_NL_df['City'] = city_field_money_NL_df['City'].apply(
    lambda x: x if x in filtered_cities_NL else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_NL_df = city_field_money_NL_df.groupby(['City', 'Field'], as_index=False).sum()

In [106]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_NL_df['City'].unique().tolist() + city_field_money_NL_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_NL_df['City'].nunique(),
    'target': city_field_money_NL_df['City'].unique(),
    'value': city_field_money_NL_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_NL_df['City'],
    'target': city_field_money_NL_df['Field'],
    'value': city_field_money_NL_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [107]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of NL (City ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [108]:
# Filter for NL
df_NL = df[df['country'] == 'NL']

# Get money percentages per city of NL and field
total_money_NL = df_NL['ecMaxContribution'].sum()
city_money_NL = (df_NL.groupby('city')['ecMaxContribution'].sum() / total_money_NL) * 100
city_field_money_NL = (df_NL.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_NL) * 100
field_activity_money_NL = (df_NL.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_NL) * 100
city_field_activity_money_NL = (df_NL.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_NL) * 100

In [109]:
# Convert the city series to a dataframe
city_money_NL_df = city_money_NL.reset_index()
city_money_NL_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_NL = city_money_NL[city_money_NL > 3]  
filtered_cities_NL = filtered_city_money_NL.index

# Convert the city-field series to a dataframe
city_field_money_NL_df = city_field_money_NL.reset_index()
city_field_money_NL_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_NL_df['City'] = city_field_money_NL_df['City'].apply(
    lambda x: x if x in filtered_cities_NL else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_NL_df = city_field_money_NL_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_NL_df = field_activity_money_NL.reset_index()
field_activity_money_NL_df.columns = ['Field', 'ActivityType', 'Money']

In [110]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_NL_df['City'].unique().tolist() + city_field_money_NL_df['Field'].unique().tolist() + field_activity_money_NL_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_NL_df['City'].nunique(),
    'target': city_field_money_NL_df['City'].unique(),
    'value': city_field_money_NL_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_NL_df['City'],
    'target': city_field_money_NL_df['Field'],
    'value': city_field_money_NL_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_NL_df['Field'],
    'target': field_activity_money_NL_df['ActivityType'],
    'value': field_activity_money_NL_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [111]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of NL (City ➝ Field ➝ Activity Type)", font_size=10)
fig.show()

## UK

### City ➝ Field

In [ ]:
# Filter for UK
df_UK = df[df['country'] == 'UK']

# Get money percentages per city of UK and field
total_money_UK = df_UK['ecMaxContribution'].sum()
city_money_UK = (df_UK.groupby('city')['ecMaxContribution'].sum() / total_money_UK) * 100
city_field_money_UK = (df_UK.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_UK) * 100

In [135]:
# Convert the city series to a dataframe
city_money_UK_df = city_money_UK.reset_index()
city_money_UK_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_UK = city_money_UK[city_money_UK > 3]  
filtered_cities_UK = filtered_city_money_UK.index

# Convert the city-field series to a dataframe
city_field_money_UK_df = city_field_money_UK.reset_index()
city_field_money_UK_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_UK_df['City'] = city_field_money_UK_df['City'].apply(
    lambda x: x if x in filtered_cities_UK else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_UK_df = city_field_money_UK_df.groupby(['City', 'Field'], as_index=False).sum()

In [136]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_UK_df['City'].unique().tolist() + city_field_money_UK_df['Field'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_UK_df['City'].nunique(),
    'target': city_field_money_UK_df['City'].unique(),
    'value': city_field_money_UK_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_UK_df['City'],
    'target': city_field_money_UK_df['Field'],
    'value': city_field_money_UK_df['Money']
})

# Map source and target columns to indices in the labels list
links = pd.concat([links1, links2], ignore_index=True)
links['source'] = links['source'].map(label_to_index)
links['target'] = links['target'].map(label_to_index)

In [137]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of UK (City ➝ Field)", font_size=10)
fig.show()

### City ➝ Field ➝ Activity Type

In [114]:
# Filter for UK
df_UK = df[df['country'] == 'UK']

# Get money percentages per city of UK and field
total_money_UK = df_UK['ecMaxContribution'].sum()
city_money_UK = (df_UK.groupby('city')['ecMaxContribution'].sum() / total_money_UK) * 100
city_field_money_UK = (df_UK.groupby(['city', 'field'])['ecMaxContribution'].sum() / total_money_UK) * 100
field_activity_money_UK = (df_UK.groupby(['field', 'activityType'])['ecMaxContribution'].sum() / total_money_UK) * 100
city_field_activity_money_UK = (df_UK.groupby(['city', 'field', 'activityType'])['ecMaxContribution'].sum() / total_money_UK) * 100

In [115]:
# Convert the city series to a dataframe
city_money_UK_df = city_money_UK.reset_index()
city_money_UK_df.columns = ['City', 'Money']

# Filter for cities with more than 3% of total money
filtered_city_money_UK = city_money_UK[city_money_UK > 3]  
filtered_cities_UK = filtered_city_money_UK.index

# Convert the city-field series to a dataframe
city_field_money_UK_df = city_field_money_UK.reset_index()
city_field_money_UK_df.columns = ['City', 'Field', 'Money']

# Classify cities not in the filtered list as 'Other'
city_field_money_UK_df['City'] = city_field_money_UK_df['City'].apply(
    lambda x: x if x in filtered_cities_UK else 'Other'
)

# Regroup after merging to 'Other'
city_field_money_UK_df = city_field_money_UK_df.groupby(['City', 'Field'], as_index=False).sum()

# Convert the field-activity series to a dataframe
field_activity_money_UK_df = field_activity_money_UK.reset_index()
field_activity_money_UK_df.columns = ['Field', 'ActivityType', 'Money']

In [116]:
# Create a list of unique labels
labels = ['ecMaxContribution'] + city_field_money_UK_df['City'].unique().tolist() + city_field_money_UK_df['Field'].unique().tolist() + field_activity_money_UK_df['ActivityType'].unique().tolist()
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Create two new DataFrames with the specified structure
# 1: ecMaxContribution ➝ City
links1 = pd.DataFrame({
    'source': ['ecMaxContribution'] * city_field_money_UK_df['City'].nunique(),
    'target': city_field_money_UK_df['City'].unique(),
    'value': city_field_money_UK_df.groupby('City')['Money'].sum().values
})

# 2: City ➝ Field
links2 = pd.DataFrame({
    'source': city_field_money_UK_df['City'],
    'target': city_field_money_UK_df['Field'],
    'value': city_field_money_UK_df['Money']
})

# 3: Field ➝ ActivityType
links3 = pd.DataFrame({
    'source': field_activity_money_UK_df['Field'],
    'target': field_activity_money_UK_df['ActivityType'],
    'value': field_activity_money_UK_df['Money']
})

# Map source and target columns to indices in the labels list
links1['source'] = links1['source'].map(label_to_index)
links1['target'] = links1['target'].map(label_to_index)

links2['source'] = links2['source'].map(label_to_index)
links2['target'] = links2['target'].map(label_to_index)

links3['source'] = links3['source'].map(label_to_index)
links3['target'] = links3['target'].map(label_to_index)

# Combine all the links
links = pd.concat([links1, links2, links3], ignore_index=True)

In [117]:
# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="blue"
    ),
    link=dict(
        source=links['source'],
        target=links['target'],
        value=links['value']
    )
)])

fig.update_layout(title_text="Sankey Diagram of ecMaxContribution of UK (City ➝ Field ➝ Activity Type)", font_size=10)
fig.show()